# Classification using neural networks

We study here the performance of feedforward neural networks to classify digits in the MNIST set. 

**There are 2 questions to answer.**

In [ ]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

# set up the random number generator: given seed for reproducibility, None otherwise
# (see https://numpy.org/doc/stable/reference/random/generator.html#numpy.random.default_rng)
my_seed = 1
rng = np.random.default_rng(seed=my_seed) 

## Construction of the dataset 

We use the MNIST dataset already encountered in various hands-on sessions. We download it through PyTorch here (note: I had to run "jupyter nbextension enable --py widgetsnbextension" in the command line for this to work; maybe also "pip3 install --upgrade ipywidgets").

In [ ]:
full_train_dataset = torchvision.datasets.MNIST(root='./data',train=True,transform=transforms.ToTensor(),download=True)
test_dataset = torchvision.datasets.MNIST(root='./data',train=False,transform=transforms.ToTensor(),download=True)

We first explore the format of the dataset.

In [ ]:
print(full_train_dataset)
print("\nformat of train data:",full_train_dataset.data.dtype)
print(full_test_dataset)
print("\nformat of test data:",full_train_dataset.data.dtype)

We next construct a training set and a validation set out of the full training set. In order to make the training faster, we can reduce the size of the dataset if we want.

In [ ]:
reduced_length = 50000
validation_length = 10000
# check whether we go out of bounds in terms of use of training data points
assert(reduced_length+validation_length<=full_train_dataset.data.shape[0])
# we can then extract the corresponding indices
indices_train = range(reduced_length)
indices_validation = range(reduced_length,reduced_length+validation_length)
train_dataset = torch.utils.data.Subset(full_train_dataset, indices=indices_train)
validation_dataset = torch.utils.data.Subset(full_train_dataset, indices=indices_validation)

We can plot the first elements of the resulting data set in order to see what they looks like.

In [ ]:
plt.figure(figsize=(10, 4))

picture_loader = torch.utils.data.DataLoader(dataset=validation_dataset,batch_size=25,shuffle=True)
examples = iter(picture_loader)
example_data, example_labels = examples.next()

for i in range(10):
    plt.subplot(2, 5, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    # color map = binary, other choices here https://matplotlib.org/stable/tutorials/colors/colormaps.html
    plt.imshow(example_data[i][0].reshape(28,28), cmap=plt.cm.binary)     
    plt.title(example_labels[i].item())
plt.show()

## Introducing neural networks

We first introduce the class of neural networks we consider. Activation functions and topology can be modified.

In [ ]:
class NeuralNet(nn.Module):
    
    def __init__(self, input_size, hidden_size, num_classes):
         super(NeuralNet, self).__init__()
         self.input_size = input_size
         self.relu = nn.ReLU()
         self.layer_1 = nn.Linear(input_size, hidden_size)
         self.layer_2 = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
         out = self.layer_1(x)
         out = self.relu(out)
         out = self.layer_2(out)
         # no activation and no softmax at the end since the loss function requires only unnormalized logits
         return out

We next specify the parameters and loss function. See https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html for the cross entropy.

In [ ]:
#--- construction of the model: from input_size to num_classes with one hidden layer --- 
input_size = 784
hidden_size = ## TO COMPLETE
num_classes = 10
model = NeuralNet(input_size, hidden_size, num_classes)

#--- Loss and optimizer ---
num_epochs = ## TO COMPLETE
batch_size = ## TO COMPLETE
learning_rate = ## TO COMPLETE
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

#--- data --- 
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
validation_loader = torch.utils.data.DataLoader(dataset=validation_dataset,batch_size=validation_length,shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,batch_size=test_dataset.data.shape[0],shuffle=True)

We can check that the test and validation sets consist of a single batch.

In [ ]:
for i, (images, labels) in enumerate(validation_loader):
    print('validation:',images.shape)
for i, (images, labels) in enumerate(test_loader):
    print('test:',images.shape)

**Question 1.** Perform the training. What accuracy can you get?

In [ ]:
early_stopping_steps = ## TO COMPLETE
min_validation_loss = ## TO COMPLETE
nb_steps_where_test_loss_did_not_decrease = 0

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
         # original shape [batch_size, 1, 28, 28], resized to [batch_size, 784]
         images = images.reshape(-1, 28*28)
         # Forward pass
         outputs = model(images)
         loss = loss_function(outputs, labels)
         # Backward and optimize
         optimizer.zero_grad()
         loss.backward()
         optimizer.step()
    # validation loss for early stopping
    nb_steps_where_test_loss_did_not_decrease += 1
    if (nb_steps_where_test_loss_did_not_decrease > early_stopping_steps):
        print('The test loss no longer decreases... STOP')
        break
    for j, (validation_images, validation_labels) in enumerate(validation_loader):
         validation_loss = loss_function(model(validation_images.reshape(-1, 28*28)),validation_labels)
    if (validation_loss < min_validation_loss):
        min_validation_loss = validation_loss
        nb_steps_where_test_loss_did_not_decrease = 0
        print(f'-- Better model // saving')
        torch.save(model, 'best_model')    
    print(f'Epoch [{epoch+1}/{num_epochs}], Train loss: {loss.item():.4f}, Validation loss: {validation_loss.item():.4f}')

We can then test the quality of the predictions made by the network.

In [ ]:
del model
model = torch.load('best_model')

# no gradient needed for evaluation
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs.data, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
acc = 100.0 * n_correct / n_samples
print(f'Accuracy of the network on the 10000 test images: {acc} %')

**Question 2.** Try to improve the performance by various means (adding an extra hidden layer, regularizing with dropout, changing the optimization algorithm, etc). What is the best test accuracy you can achieve?